# Workshop DL01 - Intro to Deep Learning

### Setup

#### Installing TensorFlow
1. Open the Anaconda3 Terminal
2. Execute `pip install --upgrade tensorflow`

#### Getting the MNIST Data
1. Go to https://www.kaggle.com/c/digit-recognizer/data
2. Download "train.csv" and "test.csv", put these into the same folder as this notebook

# Deep Neural Network Image Classifier

**Changes from v1**
* Added a third FC layer, identifcal to the second
* Changed the cost function to softmax_cross_entropy_with_logits; this required transforming the input labels to one hot
* Removed the learning rate in the AdamOptimizer, defaults to 0.001

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np

/Users/evan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [41]:
# read-in raw data
data = pd.read_csv("datasets/mnist_train.csv")

# mask for splitting data into train and test sets
mask = np.random.rand(len(data)) < 0.90

# training data, features (X) and labels (y)
train_X = data[mask].drop("label", axis=1)
train_y = data.label[mask]

# test data, features (X) and labels (y)
test_X = data[~mask].drop("label", axis=1)
test_y = data.label[~mask]

In [42]:
# helper functions for defining weights/biases in tensorflow
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
    return tf.Variable(tf.constant(0.1), shape)

In [43]:
# (1) define network architecture
with tf.name_scope("input"):
    x = tf.placeholder(tf.float32, [None, 784], name="features")
    y = tf.placeholder(tf.int64, [None], name="labels")
    y_one_hot = tf.one_hot(indices=y, depth=10)

with tf.name_scope("fc-layer-1"):
    w1 = weight_variable([784, 500])
    b1 = bias_variable([500])
    fc1 = tf.nn.relu(tf.matmul(x, w1) + b1)

with tf.name_scope("fc-layer-2"):
    w2 = weight_variable([500, 500])
    b2 = bias_variable([500])
    fc2 = tf.nn.relu(tf.matmul(fc1, w2) + b2)
    
with tf.name_scope("fc-layer-3"):
    w3 = weight_variable([500, 500])
    b3 = bias_variable([500])
    fc3 = tf.nn.relu(tf.matmul(fc2, w3) + b3)
    
with tf.name_scope("output"):
    w4 = weight_variable([500, 10])
    b4 = bias_variable([10])
    y_ = tf.matmul(fc3, w4) + b4

In [44]:
# (2) define loss function
with tf.name_scope("loss"):
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=y_, labels=y_one_hot)
    )

In [45]:
# (3) define optimizer
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer().minimize(
        cross_entropy
    )

In [46]:
# aside: evaluation operations
with tf.name_scope("accuracy"):
    correct = tf.equal(tf.argmax(y_, 1), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [50]:
# (4) train network
with tf.Session() as sess:
    
    # initialize all our operations
    sess.run(tf.global_variables_initializer())
    
    # train the network
    print("==================================================")
    for epoch in range(50):
        
        # Note: usually you have to chunk through the data in another
        #       for loop; the data here is small though, so you might not
        #       have to.
        _, epoch_loss = sess.run([optimizer, cross_entropy],
                feed_dict={x: train_X, y: train_y})
        
        print("Epoch " + str(epoch+1) + " completed, loss = "
              + str(epoch_loss))
        
        # evaluate on test data
        score = sess.run([accuracy],
                feed_dict={x: test_X, y: test_y})
        print("Test accuracy = " + str(score))
        
        print("==================================================")

Epoch 1 completed, loss = 881.16766
Test accuracy = [0.19347826]
Epoch 2 completed, loss = 516.1829
Test accuracy = [0.33333334]
Epoch 3 completed, loss = 518.0711
Test accuracy = [0.42536232]
Epoch 4 completed, loss = 499.1371
Test accuracy = [0.526087]
Epoch 5 completed, loss = 408.74295
Test accuracy = [0.5821256]
Epoch 6 completed, loss = 317.60574
Test accuracy = [0.6263285]
Epoch 7 completed, loss = 219.16394
Test accuracy = [0.6611111]
Epoch 8 completed, loss = 135.15935
Test accuracy = [0.70821255]
Epoch 9 completed, loss = 80.794846
Test accuracy = [0.74396133]
Epoch 10 completed, loss = 67.62097
Test accuracy = [0.7504831]
Epoch 11 completed, loss = 66.16924
Test accuracy = [0.7504831]
Epoch 12 completed, loss = 65.22422
Test accuracy = [0.73478264]
Epoch 13 completed, loss = 72.17933
Test accuracy = [0.7326087]
Epoch 14 completed, loss = 72.17477
Test accuracy = [0.7545894]
Epoch 15 completed, loss = 59.44088
Test accuracy = [0.797343]
Epoch 16 completed, loss = 44.15338
Tes